# Jaguar Vector Store

This document demonstrates llama_index working with Jaguar vector store.

1) It is a distributed vector database that can store large number of vectors.
2) The ZeroMove feature of JaguarDB enables instant horizontal scalability.
3) It works with multimodal data including vector embeddings, text, images, videos, PDFs, audio, time series, and geospatial data. 
4) The all-master architecture allows both parallel reads and writes.
5) Its anomaly detection capabilities can distinguish outliers in the dataset.
6) The RAG support can combine LLMs and proprietary and real-time data.
7) Sharing of metadata across multiple vector indexes improves data consistency.
8) Distance metrics include Euclidean, Cosine, InnerProduct, Manhatten, Chebyshev, Hamming, Jeccard, and Minkowski.
9) Similarity search can be performed with time cutoff and time decay effects.

## Prerequisites

There are two requirements for running the examples in this file.

You must install and set up the JaguarDB server and its HTTP gateway server. 
Please refer to the instructions:

    http://www.jaguardb.com

You must install packages llama-index and jaguardb-http-client.

    !pip install -U llama-index
    !pip install -U jaguardb-http-client

    

In [ ]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import OpenAIEmbedding
from llama_index.vector_stores.jaguar import JaguarVectorStore
from jaguardb_http_client.JaguarHttpClient import JaguarHttpClient

### instantiate a jaguar vector store client object
url = "http://127.0.0.1:8080/fwww/"
pod = "vdb"
store = "llamaindex_jaguar_store"
vector_index = "v"
# vector_type = "cosine_fraction_float"
vector_type = "cosine_fraction_short"  # half memory usage
# vector_type = "cosine_fraction_byte"
vector_dimension = 1536  # per OpenAIEmbedding model
jaguarstore = JaguarVectorStore(
    pod,
    store,
    vector_index,
    vector_type,
    vector_dimension,
    url,
)

### connect to jaguar server
jaguarstore.login()

### create a vector index store
"""
Create a vector with vector index 'v' and 'v:text' with size 1024 bytes
to hold text and metadata author and category
"""
metadata_str = "author char(32), category char(16)"
text_size = 1024
jaguarstore.create(metadata_str, text_size)

### load paul gram example documents
documents = SimpleDirectoryReader("../data/paul_graham/").load_data()

### make a storage context using our vector store
storage_context = StorageContext.from_defaults(vector_store=jaguarstore)

### have a service context using the openai embedding model
embed_model = OpenAIEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model)

### make an index with the documents,storage context, and service context
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

### print number of documents in jaguar vector store
num = jaguarstore.count()
print(f"There are {num} documents in jaguar vector store")

### get the query engine and ask it some questions
query_engine = index.as_query_engine()
q = "What did the author do growing up?"
print(f"Question: {q}")
response = query_engine.query(q)
print(f"Answer: {str(response)}")

q = "What did the author do after his time at Viaweb?"
print(f"Question: {q}")
response = query_engine.query(q)
print(f"Answer: {str(response)}")

### remove all the data in the vector store if you want
jaguarstore.clear()

### delete the whole vector in the database if you want
jaguarstore.drop()

### disconnect from jaguar server and cleanup resources
jaguarstore.logout()